In [ ]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/QuantumFlow'
except:
    project_path = os.path.expanduser('~/QuantumFlow')

In [ ]:
os.chdir(project_path)
sys.path.append(project_path)

import numpy as np
import tensorflow as tf
import tree

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import quantumflow

experiment = 'xdiff'
run_name = 'default'
epoch = None

preview = 5

base_dir = os.path.join(project_path, "experiments", experiment)
params = quantumflow.utils.load_yaml(os.path.join(base_dir, 'hyperparams.yaml'))[run_name]
run_dir = os.path.join(base_dir, run_name)

In [ ]:
dataset_validate = quantumflow.instantiate(params['dataset_validate'], run_dir=run_dir)
dataset_validate.build()

In [ ]:
_ = quantumflow.get_class(params['model']['class'])
model = tf.keras.models.load_model(os.path.join(run_dir, 'saved_model'))
if epoch is not None: _ = model.load_weights(os.path.join(run_dir, params['checkpoint']['filename'].format(epoch=epoch)))
model.summary()

In [ ]:
@tf.function
def predict_fn(features_batch):
    density = tf.nest.flatten(features_batch)
    
    with tf.GradientTape() as tape:
        tape.watch(density)
        outputs_batch = model(density)
    
    outputs_batch['derivative'] = 1/dataset_validate.h*tape.gradient(outputs_batch['kinetic_energy'], density)[0]
    return outputs_batch

def predict(features, batch_size=None):
    if batch_size is None:
        return tree.map_structure(lambda out: out.numpy(), predict_fn(features))
    else:
        outputs = []
        dataset_size = tree.flatten(features)[0].shape[0]
        steps = -(-dataset_size//batch_size)
        print_steps = max(1, steps//100)
        print('/', dataset_size)
        for i in range(steps):
            if i % print_steps == 0: print(i*batch_size, end=' ')
            features_batch = tree.map_structure(lambda inp: inp[i*batch_size:(i+1)*batch_size], features)
            outputs.append(predict_fn(features_batch))
        print()
        return tree.map_structure(lambda *outs: np.concatenate(outs, axis=0), *outputs)

In [ ]:
targets_pred = predict(dataset_validate.features, params['dataset_validate'].get('max_batch_size', 1))

In [ ]:
targets_pred['kinetic_energy'][:preview]

In [ ]:
dataset_validate.targets['kinetic_energy'][:preview]

In [ ]:
kinetic_energy_err = targets_pred['kinetic_energy'] - dataset_validate.targets['kinetic_energy'][:len(targets_pred['kinetic_energy'])]

In [ ]:
kcalmol_per_hartree = 627.5094738898777
np.mean(np.abs(kinetic_energy_err))*kcalmol_per_hartree

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(dataset_validate.x, dataset_validate.targets['kinetic_energy_density'][:preview, :].transpose(), 'k:')
plt.plot(dataset_validate.x, targets_pred['kinetic_energy_density'][:preview, :].transpose())
plt.show()

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(dataset_validate.x, dataset_validate.targets['kinetic_energy_density'][:preview, :].transpose() - targets_pred['kinetic_energy_density'][:preview, :].transpose())
plt.show()

In [ ]:
targets_pred['derivative'].shape

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(dataset_validate.x, dataset_validate.derivative[:preview, :].transpose(), 'k:')
plt.plot(dataset_validate.x, targets_pred['derivative'][:preview, :].transpose())
plt.show()

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')['peak']/1024**3